### 1. lib import

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver

from urllib.request import urlopen
from urllib.error   import HTTPError
from urllib.request import URLError

import numpy as np
import pandas as pd

import json, re
import pymongo

### 2. DB connection

In [4]:
conn = pymongo.MongoClient()
movieDB = conn.movieDB
movieCollection = movieDB.movieCollection

### 3. request : http://www.cine21.com/rank/person

In [6]:
path = '../driver/chromedriver.exe'
url = 'http://www.cine21.com/rank/person'
driver = webdriver.Chrome(path)
driver.get(url)

### 4. Actor name

In [9]:
# //*[@id="rank_holder"]/ul/li[1]/div/a
# //*[@id="rank_holder"]/ul/li[2]/div/a

for i in range(1, 8) :
    name = driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}]/div/a'.format(i))
    print(re.sub('\(\w*\)', '', name.text))

김영광
이선빈
한예리
윤여정
김강우
유인나
유연석


### 5. 정보 가져오기
- 배우이름, 흥행지수, 출연영화[], 랭킹, 원어명, 다른이름, 직업, 생년월일, 성별, 홈페이지, 신장/체중, 학교, 취미, 특기...
- 배우이름, 흥행지수, 출연영화, 랭킹 -> DB입력

In [24]:
data = []
for i in range(1,8) :
    actor  = driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}]/div/a'.format(i))
    hits   = driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}]/ul[1]/li/strong'.format(i))
    movies = driver.find_elements_by_xpath('//*[@id="rank_holder"]/ul/li[{}]/ul[2]/li/a/span'.format(i))
    rank = driver.find_element_by_xpath('//*[@id="rank_holder"]/ul/li[{}]/span'.format(i))

    #print("배우이름:", re.sub('\(\w*\)', '', actor.text))
    #print("랭킹:",rank.text )
    #print("흥행지수:", int(hits.get_attribute('innerHTML').replace(',', '')))
    movie_title_list = list()
    for movie_title in movies:
        movie_title_list.append(movie_title.text)
    #print("출연영화:", movie_title_list)
    data.append(["배우이름:", re.sub('\(\w*\)', '', actor.text), 
                 '랭킹:',rank.text,
                 "흥행지수:", int(hits.get_attribute('innerHTML').replace(',', '')),
                 "출연영화:", movie_title_list])
data

[['배우이름:', '김영광', '랭킹:', '1', '흥행지수:', 277566, '출연영화:', ['미션 파서블']],
 ['배우이름:', '이선빈', '랭킹:', '2', '흥행지수:', 138784, '출연영화:', ['미션 파서블']],
 ['배우이름:', '한예리', '랭킹:', '3', '흥행지수:', 113758, '출연영화:', ['미나리']],
 ['배우이름:', '윤여정', '랭킹:', '4', '흥행지수:', 94807, '출연영화:', ['미나리', '찬실이는 복도 많지']],
 ['배우이름:', '김강우', '랭킹:', '5', '흥행지수:', 36772, '출연영화:', ['새해전야']],
 ['배우이름:', '유인나', '랭킹:', '6', '흥행지수:', 32686, '출연영화:', ['새해전야']],
 ['배우이름:', '유연석', '랭킹:', '7', '흥행지수:', 28600, '출연영화:', ['새해전야']]]

In [32]:
actor_info_list = []

for obj in data :
    actor_info_list.append({
        '배우이름' : obj[1],
        '랭킹' : obj[3],
        '흥행지수' : obj[5],
        '출연영화' : obj[7]
    })
    
actor_info_list

[{'배우이름': '김영광', '랭킹': '1', '흥행지수': 277566, '출연영화': ['미션 파서블']},
 {'배우이름': '이선빈', '랭킹': '2', '흥행지수': 138784, '출연영화': ['미션 파서블']},
 {'배우이름': '한예리', '랭킹': '3', '흥행지수': 113758, '출연영화': ['미나리']},
 {'배우이름': '윤여정', '랭킹': '4', '흥행지수': 94807, '출연영화': ['미나리', '찬실이는 복도 많지']},
 {'배우이름': '김강우', '랭킹': '5', '흥행지수': 36772, '출연영화': ['새해전야']},
 {'배우이름': '유인나', '랭킹': '6', '흥행지수': 32686, '출연영화': ['새해전야']},
 {'배우이름': '유연석', '랭킹': '7', '흥행지수': 28600, '출연영화': ['새해전야']}]

In [33]:
movieCollection.insert_many(actor_info_list)

In [34]:
movieCollection.find_one()

{'_id': ObjectId('604b1b7bebb6ce2736fc1a07'),
 '배우이름': '김영광',
 '랭킹': '1',
 '흥행지수': 277566,
 '출연영화': ['미션 파서블']}

### 6. 크롤링 정보 저장

In [49]:
with open ('./actor_info_list.json', mode='w', encoding='utf-8') as file :
    actor_info_list = []
    for obj in data :
        actor_info_list.append({
            '배우이름' : obj[1],
            '랭킹' : obj[3],
            '흥행지수' : obj[5],
            '출연영화' : obj[7]
        })
        print(json.dumps(actor_info_list, ensure_ascii=False, indent=4))
    json.dump(actor_info_list, file, indent=4, ensure_ascii=False)

[
    {
        "배우이름": "김영광",
        "랭킹": "1",
        "흥행지수": 277566,
        "출연영화": [
            "미션 파서블"
        ]
    }
]
[
    {
        "배우이름": "김영광",
        "랭킹": "1",
        "흥행지수": 277566,
        "출연영화": [
            "미션 파서블"
        ]
    },
    {
        "배우이름": "이선빈",
        "랭킹": "2",
        "흥행지수": 138784,
        "출연영화": [
            "미션 파서블"
        ]
    }
]
[
    {
        "배우이름": "김영광",
        "랭킹": "1",
        "흥행지수": 277566,
        "출연영화": [
            "미션 파서블"
        ]
    },
    {
        "배우이름": "이선빈",
        "랭킹": "2",
        "흥행지수": 138784,
        "출연영화": [
            "미션 파서블"
        ]
    },
    {
        "배우이름": "한예리",
        "랭킹": "3",
        "흥행지수": 113758,
        "출연영화": [
            "미나리"
        ]
    }
]
[
    {
        "배우이름": "김영광",
        "랭킹": "1",
        "흥행지수": 277566,
        "출연영화": [
            "미션 파서블"
        ]
    },
    {
        "배우이름": "이선빈",
        "랭킹": "2",
        "흥행지수": 138784,
        "출연영화": [
      

In [50]:
with open('./actor_info_list.json', mode='r', encoding='utf-8')as file :
    json_data = json.load(file)

In [51]:
json_data

[{'배우이름': '김영광', '랭킹': '1', '흥행지수': 277566, '출연영화': ['미션 파서블']},
 {'배우이름': '이선빈', '랭킹': '2', '흥행지수': 138784, '출연영화': ['미션 파서블']},
 {'배우이름': '한예리', '랭킹': '3', '흥행지수': 113758, '출연영화': ['미나리']},
 {'배우이름': '윤여정', '랭킹': '4', '흥행지수': 94807, '출연영화': ['미나리', '찬실이는 복도 많지']},
 {'배우이름': '김강우', '랭킹': '5', '흥행지수': 36772, '출연영화': ['새해전야']},
 {'배우이름': '유인나', '랭킹': '6', '흥행지수': 32686, '출연영화': ['새해전야']},
 {'배우이름': '유연석', '랭킹': '7', '흥행지수': 28600, '출연영화': ['새해전야']}]

### 7. 첫 페이지의 정보저장이 완료되면 페이징처리 후 저장

In [52]:
from time import time, sleep
from random import randint
from IPython.core.display import clear_output
from requests import get

In [ ]:
for page in range(1, 53):
    url = 'https://www.ildong.com/kor/product/list.id?page='+str(page)+'&halt=&prdDisease=&prdCategory=5&searchVal=&searchOption=0'
    #print(url)
    driver.get(url)
    prList = driver.find_element_by_class_name('prList')
    tags = prList.find_elements_by_tag_name('a')
    
    for idx in range(0, 9):
        #print(tags[idx])
        tags[idx].click()
        prd_name = driver.find_element_by_class_name('detailCnt2')
        print(prd_name.text)
        driver.back()

        prList = driver.find_element_by_class_name('prList')
        tags = prList.find_elements_by_tag_name('a')
        #sleep(2)